# Web2cit: up-to-date Citoid gap figures

This notebook describes how to retrieve the source code and the [Citation templates](https://en.wikipedia.org/wiki/Wikipedia:Citation_templates) of the references of a sample of Wikipedia articles to compare them with the results of the [Citoid](https://www.mediawiki.org/wiki/Citoid) API for the same references.

To follow along, we recommend running the script portions piecemeal, in order.

__Author:__

* Nidia Hernández, [nidiahernandez@conicet.gov.ar](mailto:nidiahy@gmail.com), CAICYT-CONICET


## Table of Contents

0. Setting Up 
1. Retrieving data from Wikipedia articles
    - 1.1. Fetching featured articles using Mediawiki's action API
    - 1.2. Retrieve featured articles data
2. Citation template metadata extraction
3. Querying Citoid API
4. Evaluating Citoid results
5. Visualizing results
    - 5.1. Tabular representation
    - 5.2. Other representations ...
6. ...
    - 6.1. ...
    - 6.2. ...




## 0. Setting Up

Before we get started, let's install and import the libraries that we will need.

In [38]:
import re
import os
import requests
import urllib
import pandas as pd
from operator import itemgetter
import json
import gzip
from pprint import pprint
from tqdm import tqdm
import mwparserfromhell

Here we set the global parameters:

In [2]:
HEADER={'User-Agent': 'http://caicyt-conicet.gov.ar/; mailto:nidiahernandez@conicet.gov.ar'}

#CITOID_API_URL = f'https://{lang}.wikipedia.org/api/rest_v1/data/citation/mediawiki/'

## 1. Retrieving data from Wikipedia articles

In this section, we will fetch all the featured articles from a selection of Wikipedias ir order to find:

- the citation templates used on each article
- the URLs of the citations

The selected Wikipedias are the following:

- [English Wikipedia](https://en.wikipedia.org/wiki/Category:Featured_articles): ~6k featured articles
- [Spanish Wikipedia](https://es.wikipedia.org/wiki/Categor%C3%ADa:Wikipedia:Art%C3%ADculos_destacados): ~1.2k artículos destacados
- [French Wikipedia](https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:Article_de_qualit%C3%A9): ~2k articles de qualité
- [Portuguese Wikipedia](https://pt.wikipedia.org/wiki/Categoria:!Artigos_destacados): ~1.3k artigos destacados

Some general information about featured articles in all Wikipedias: https://meta.wikimedia.org/wiki/Wikipedia_featured_articles (out of date).

#### Featured content endpoint in Mediawiki's API

There is a specific endpoint for featured content in Mediawiki's API:

https://api.wikimedia.org/feed/v1/wikipedia/{lang}/featured/

For example, the request https://api.wikimedia.org/feed/v1/wikipedia/en/featured/2021/12/06 returns today's featured article (tfa) for December 6th 2021 in English Wikipedia.

Unfortunatatly, this is not avalilable for all the Wikipedias of our interest because "not all Wikipedias are integrated into the Feed endpoint" ([Feed endpoint doc](https://api.wikimedia.org/wiki/API_reference/Feed/Featured_content)). This way, the request https://api.wikimedia.org/feed/v1/wikipedia/fr/featured/2021/12/06 does not return tfa info. Same for Spanish and Portuguese. For this reason, we do not use this specific endpoint.

### 1.1 Fetching featured articles using Mediawiki's action API

We will use each [Wikipedias' action API](https://www.mediawiki.org/wiki/API:Main_page) to retrive:

1. the list of all featured articles for each language
2. the wikicode and some metadata of each featured article

The action API has the following URL:

`https://{lang}.wikipedia.org/w/api.php?`

The following request, for example, retrieves the first 15 featured articles for the category "Articles de qualité" in French Wikipedia:

In [3]:
title = urllib.parse.unquote("Cat%C3%A9gorie:Article_de_qualit%C3%A9") # category name for featured articles in French Wikipedia

PARAMS = {
    "action": "query",
    "list": "categorymembers",
    "cmtitle": title,
    "cmlimit": 15,
    "format": "json"
}

response = requests.get(
    url='https://fr.wikipedia.org/w/api.php?', 
    params=PARAMS
)

response_data = response.json()

There is a max of 500 per request. In order to retrieve all the members of the category, we should add the parameter `cmcontinue` to our request as follows: we get the value of `cmcontinue` from the response of the first request and pass it to the following request. This repeats until the end of the list. See API doc for more details on listing category members: https://www.mediawiki.org/wiki/API:Categorymembers.

In [4]:
response_data['continue']['cmcontinue'] # here is the reference of the starting point for the next request

'page|2a2e38324232443a30324e01428844880901dc10|52519'

In the response, we obtain a list showing the pageid and the title of each article:

In [5]:
response_data['query']['categorymembers']

[{'pageid': 507899, 'ns': 4, 'title': 'Wikipédia:Articles de qualité'},
 {'pageid': 9253139, 'ns': 0, 'title': '1 000 kilomètres de Spa 2009'},
 {'pageid': 10267261, 'ns': 0, 'title': '6 Heures du Castellet 2011'},
 {'pageid': 91996, 'ns': 0, 'title': '32X'},
 {'pageid': 605584, 'ns': 0, 'title': 'Les 101 Dalmatiens (film, 1961)'},
 {'pageid': 5008651, 'ns': 0, 'title': 'A Different Corner'},
 {'pageid': 639219, 'ns': 0, 'title': "A Hard Day's Night (album)"},
 {'pageid': 6672844, 'ns': 0, 'title': "À l'Olympia (album d'Alan Stivell)"},
 {'pageid': 124634, 'ns': 0, 'title': 'À la croisée des mondes'},
 {'pageid': 1486250, 'ns': 0, 'title': "L'Abbaye de Northanger"},
 {'pageid': 11085052, 'ns': 0, 'title': 'Abbaye Saint-Paul de Cormery'},
 {'pageid': 1210952, 'ns': 0, 'title': 'Abbaye Saint-Victor de Marseille'},
 {'pageid': 4293975, 'ns': 0, 'title': "Parc national d'Abisko"},
 {'pageid': 1415801, 'ns': 0, 'title': 'Acanthaster planci'},
 {'pageid': 3906668,
  'ns': 0,
  'title': 'Acci

If the item is a featured article, `ns` (namespace) is `0`. So, to avoid retriving other items (meta-pages about other categories, for example) elements having other values for this key must be discarded (cf. Retrieve article data below).

Let's request all the featured articles for each language and dump the response to a json:

In [6]:
# the name of the category in each language
catname_bylang = {
    'en': 'Category:Featured_articles',
    'es': 'Categor%C3%ADa:Wikipedia:Art%C3%ADculos_destacados',
    'fr': 'Cat%C3%A9gorie:Article_de_qualit%C3%A9',
    'pt': 'Categoria:!Artigos_destacados',
}

In [7]:
for lang in tqdm(catname_bylang, desc='Retrieving featured articles'):
    
    MEDIAWIKI_API_URL = f'https://{lang}.wikipedia.org/w/api.php?'
    
    category_title = urllib.parse.unquote(catname_bylang[lang])
    cmcont = "start"
    i = 0
    
    while (cmcont == "start") or cmcont.startswith('page'):
        i += 1
        filename = f'featured-articles/{lang}-{i:02}.json'
        
        if os.path.isfile(filename):
            with open(filename) as fi:
                featured_art_json = json.load(fi)
                if 'continue' in featured_art_json:
                    cmcont = featured_art_json['continue']['cmcontinue']
                else:
                    cmcont = "end"

            continue
        
        PARAMS = {
            "action": "query",
            "list": "categorymembers",
            "cmtitle": category_title,
            "cmlimit": "max", 
            "format": "json",
            "curtimestamp": True
        }
            
        if cmcont:
            PARAMS["cmcontinue"]=cmcont

        response = requests.get(
            url=MEDIAWIKI_API_URL, 
            params=PARAMS,
            headers=HEADER
        )

        data = response.json()

        if response.status_code == 200:

            if 'continue' in data:
                cmcont = data['continue']['cmcontinue']
            
            else:
                cmcont='categoryend'

            with open(filename, 'w') as fo:
                fo.write(json.dumps(data))

Retrieving featured articles: 100%|██████████| 4/4 [00:00<00:00, 176.40it/s]


### 1.2 Retrieve featured articles data

Now that we have the pageids of all the featured articles, we can use them to retrieve the URL, the content and the id of the last edition (`revid`). First, we load the information from the json and we build a dataframe where we will store the data for each featured article:

In [49]:
# Get the information of the featured articles and build a dataframe
dfs = []

for fname in sorted(os.listdir('featured-articles')):
    if fname.endswith('.json'):
        
        wikilang = fname.split('-')[0]
        
        with open(f'featured-articles/{fname}') as fi:
            featured_arts = json.load(fi)
            
        df = pd.DataFrame(featured_arts['query']['categorymembers'])
        df['wiki_lang'] = wikilang
        df = df[df['ns'] == 0] # discard items that are not featured articles
        df.drop('ns', axis='columns', inplace = True)

        dfs.append(df)
    
articles_data = pd.concat(dfs, ignore_index= True)
articles_data.rename(columns={'title':'article_title'}, inplace = True)
articles_data.drop_duplicates(inplace = True)

In [50]:
articles_data

,pageid,article_title,wiki_lang
0,33653136,? (film),en
1,1849799,0.999...,en
2,9702578,1 − 2 + 3 − 4 + ⋯,en
3,48723612,1st Cavalry Division (Kingdom of Yugoslavia),en
4,64662351,1st Missouri Field Battery,en
...,...,...,...
15065,132185,Yorkshire terrier,pt
15066,2269483,You Belong with Me,pt
15067,4779921,You Don't Know What to Do,pt
15068,1585284,You Know I'm No Good,pt


We add a new column containing the URL of each article using the information from the article title and the language of the wikipedia:

In [51]:
articles_data['article_url'] = "https://"+articles_data['wiki_lang']+".wikipedia.org/wiki/"+articles_data['article_title'].map(urllib.parse.quote)

In [52]:
articles_data

,pageid,article_title,wiki_lang,article_url
0,33653136,? (film),en,https://en.wikipedia.org/wiki/%3F%20%28film%29
1,1849799,0.999...,en,https://en.wikipedia.org/wiki/0.999...
2,9702578,1 − 2 + 3 − 4 + ⋯,en,https://en.wikipedia.org/wiki/1%20%E2%88%92%202%20%2B%203%20%E2%88%92%204%20%2B%20%E2%8B%AF
3,48723612,1st Cavalry Division (Kingdom of Yugoslavia),en,https://en.wikipedia.org/wiki/1st%20Cavalry%20Division%20%28Kingdom%20of%20Yugoslavia%29
4,64662351,1st Missouri Field Battery,en,https://en.wikipedia.org/wiki/1st%20Missouri%20Field%20Battery
...,...,...,...,...
15065,132185,Yorkshire terrier,pt,https://pt.wikipedia.org/wiki/Yorkshire%20terrier
15066,2269483,You Belong with Me,pt,https://pt.wikipedia.org/wiki/You%20Belong%20with%20Me
15067,4779921,You Don't Know What to Do,pt,https://pt.wikipedia.org/wiki/You%20Don%27t%20Know%20What%20to%20Do
15068,1585284,You Know I'm No Good,pt,https://pt.wikipedia.org/wiki/You%20Know%20I%27m%20No%20Good


We save this information to a csv file:

In [35]:
articles_data.to_csv('articles_data.csv')

Now we are ready to query Mediawiki's action API again to get the content of each featured article. 

For example, to retrieve the wikitext and the id of the last edition for the article #3906668 'Accident sur la base de Fairchild en 1994' from French Wikipedia, we can do the following request:

In [36]:
pageid = 3906668 # article from French Wikipedia 'Accident sur la base de Fairchild en 1994'

PARAMS = {
    "action": "parse",
    "pageid": pageid,
    "prop": "wikitext|revid",
    "format": "json"
}

response = requests.get(
    url='https://fr.wikipedia.org/w/api.php?', 
    params=PARAMS,
)

page_data = response.json()

In [37]:
page_data

{'parse': {'title': 'Accident sur la base de Fairchild en 1994',
  'pageid': 3906668,
  'revid': 186075124,
  'wikitext': {'*': '{{En-tête label|AdQ}}\n{{Infobox Accident de transport\n  | nom                  = Accident du B-52 sur la base de Fairchild en 1994\n  | image                = FairchildB52Crash.jpg\n  | légende              = Le B-52 sur la tranche une seconde avant de toucher le sol.\n  | date                 = {{date|24|juin|1994|en aéronautique}}\n  | phase                = Vol d\'entraînement\n  | type                 = Erreur de pilotage\n  | site                 = [[Fairchild Air Force Base]], [[Washington (État)|Washington]], [[États-Unis]]\n  | passagers            = \n  | équipage             = 4 militaires\n  | morts                = 4 militaires\n  | blessés              = \n  | survivants           = \n  | appareil             = [[Boeing B-52 Stratofortress|Boeing B-52H \'\'Stratofortress\'\']]\n  | compagnie            = [[United States Air Force]]\n  | numéro_

Let's fetch the wikitext and the id of the last edition for each featured article and save the response to a compressed file:

In [53]:
# Retrive wikitext for all featured articles

for index, row in tqdm(articles_data.iterrows(), desc='Retrieving data from articles'):

    pageid = articles_data.loc[index]['pageid']
    lang = articles_data.loc[index]['wiki_lang']
    filename = f'articles-content/{pageid}-{lang}.json.gz'
    
    MEDIAWIKI_API_URL = f'https://{lang}.wikipedia.org/w/api.php?'

    if not os.path.isfile(filename):

        PARAMS = {
            "action": "parse",
            "pageid": pageid,
            "prop": "wikitext|revid",
            "format": "json"
        }

        response = requests.get(
            url=MEDIAWIKI_API_URL, 
            params=PARAMS,
            headers=HEADER
        )

        page_data = response.json()

        if response.status_code == 200:
            with gzip.open(filename, 'w') as fo:
                fo.write(json.dumps(page_data).encode('utf-8'))
    
    else:
        continue


Retrieving data from articles: 10570it [00:06, 1640.30it/s]


## 2. Citation template metadata extraction

Now that we already have the wikitext of the articles, we will parse it in order to retrieve the references that were introduced using a citation template. In other words, we are not interested in:

1. Manually entered references (ie, which do not use citation templates)

`<ref name=Briggs>Briggs, Asa & Burke, Peter (2002) ''A Social History of the Media: from Gutenberg to the Internet'', Cambridge: Polity, pp. 15–23, 61–73.</ref>`

2. Unlinked references

```<ref name=Neeham>{{cite book |title=Paper and Printing |author=[[Tsien Tsuen-Hsuin]] |author2=[[Joseph Needham]] |series=Science and Civilisation in China|volume=5 part 1|publisher=Cambridge University Press|pages=158, 201|year=1985}}</ref> ```

Instead, we are interested in:

1. References having a URL

```<ref name="VB1992">{{cite journal|last1=Osmond|first1=Patricia J.|last2=Ulery|first2=Robert W.|date=2003|title=Sallustius|url=http://catalogustranslationum.org/PDFs/volume08/v08_sallustius.pdf#page=17|journal=[[Catalogus Translationum et Commentariorum]]|volume=8|page=199|access-date=27 August 2015}}</ref>```

2. References having a DOI (revisar si hay casos de doi sin url)

```<ref name="NH99-2362">{{cite journal|last=Holzberg|first=Niklas|date=1999|title=The Fabulist, the Scholars, and the Discourse: Aesop Studies Today|url=https://www.jstor.org/stable/30222546|journal=International Journal of the Classical Tradition|volume=6|pages=236–242|doi=10.1007/s12138-999-0004-y|jstor=30222546|access-date=31 January 2021|number=2|s2cid=195318862}}</ref>```

Therefore, we parse the page content looking only for the citation templates including a url.

We parse the wikicode using [mwparserfromhell](https://github.com/earwig/mwparserfromhell).

We are interested in the following metadata: 
1. The source type (journal, book, website, etc.)
1. The URL of the source
2. The author(s)
3. The title
4. The publishing date
5. The publishing source (publisher, location, etc.)

The names of each data may vary between templates. For instance, the publishing source is under "periodical" for the news template and under "publisher" for the books template. Or the publishing date might be called "date" or "year" in the maps template. The following spreadsheet maps the name of each parameter in the citations templates to our fieldnames:

In [54]:
mapping_sheet = f'https://docs.google.com/spreadsheets/d/1xbc3FKE0m4JQHa6WCXtBbzeJ9in8P0EQ2NF_VNsaBaM/export?gid=0&format=csv'

fieldnames_df = pd.read_csv(mapping_sheet)
fieldnames_df.rename(columns={
    "Template": "template",
    "authorLast": "author_last",
    "authorFirst": "author_first",
    "pubDate": "pub_date",
    "source (published In + published By)": "pub_source"
}, inplace=True)

fieldnames_df

,wiki_lang,template,title,author_last,author_first,pub_date,pub_source
0,en,Cite web,title,last\d*,first\d*,date,"website, publisher"
1,es,Cita web,título,apellido\d*,nombre\d*,fecha,"sitioweb, obra, publicación, editorial"
2,pt,Citar web,título,último\d*,primeiro\d*,data,"obra, publicado"
3,fr,Lien web,titre,nom\d*,prénom\d*,date,site
4,en,Cite book,title,last\d*,first\d*,date,"publication-place, location, publisher"
...,...,...,...,...,...,...,...
75,pt,Citar tese,título,último\d*,primeiro\d*,"data, ano","editora, publicado"
76,en,Cite arXiv,title,last\d*,first\d*,date,eprint
77,es,Cita arXiv,título,apellido\d*,nombre\d*,fecha,eprint
78,fr,Lien arXiv,titre,nom\d*,prénom\d*,date,eprint


In [58]:
for lang in fieldnames_df['wiki_lang'].unique():
    print(f"We have {len(fieldnames_df[fieldnames_df['wiki_lang'] == lang])} templates in {lang}")

We have 26 templates in en
We have 18 templates in es
We have 24 templates in pt
We have 12 templates in fr


In [ ]:
citations_data_dir = 'citations'
os.makedirs(citations_data_dir, exist_ok = True)

citation_nr = 0
citation_url_nr = 0
tempname_in_spreadsheet = 0
tempname_in_spreadsheet_url = 0
not_parsed = 0

for filename in tqdm(glob.glob('articles-content/*json.gz'), desc=f"Finding citations and extracting metadata"):
    citations = []
    new_filename = f"citations/{os.path.basename(filename).replace('json.gz', 'csv.gz')}"
    
    if os.path.isfile(new_filename):
        continue
    
    with gzip.open(filename) as fi:
        cont = loads(fi.read().decode('utf-8'))

    lang = os.path.basename(filename).split('-')[1].replace('.json.gz', '')
    pageid = cont['parse']['pageid'] 
    revid = cont['parse']['revid']
    article_title = cont['parse']['title']
    article_url = f'https://{lang}.wikipedia.org/wiki/{urllib.parse.quote(article_title)}'
    wikitext = cont['parse']['wikitext']['*']

    parsed_wiki = mwparserfromhell.parse(wikitext)

    for template in parsed_wiki.filter_templates():

        for index, fieldname in fieldnames_df[fieldnames_df['wiki_lang'] == lang].iterrows():

            if re.match(r'[Cc]it', str(template.name)):
                citation_nr+=1
                if template.has('url') or template.has('URL'):
                    citation_url_nr += 1
                
            if template.name.matches(fieldname['template']):
                tempname_in_spreadsheet +=1
                if template.has('url') or template.has('URL'):
                    tempname_in_spreadsheet_url += 1

                    # For the keys to appear in the right order
                    citation = {
                        "article_title": article_title,
                        "article_url": article_url,
                        "page_id": pageid,
                        "revid": revid,
                        "wiki_lang": lang,
                        "url": None,
                        "source_type": template.name.strip(),
                        "title": None,
                        "author_last": [],
                        "author_first": [],
                        "pub_date": None,
                        "pub_source": None,
                    }

                    wikicode_params = [param.name.strip() for param in template.params]

                    # Fields separados por coma
                    for key in ["title", "pub_date", "pub_source"]:
                        param_names = [name for name in fieldname[key].split(", ")
                                       if name and name in wikicode_params]

                        for param_name in param_names:
                            citation[key] = template.get(param_name).value.strip()

                    url_fieldname = "url" if "url" in wikicode_params else "URL"
                    citation["url"] = template.get(url_fieldname).value.strip()

                    # Fields con regex
                    for our_fieldname in ["author_first", "author_last"]:
                        wiki_fieldname_regex = f'{fieldname[our_fieldname]}\b'
                        matching_params = re.findall(wiki_fieldname_regex, str(template))
                        for param in matching_params:
                            citation[our_fieldname].append(template.get(param).value.strip())

                    citations.append(citation)
                
    citations_df = pd.DataFrame(citations)
    if not citations_df.empty: # Save only if parsing ok
        citations_df.to_csv(new_filename, index=False)
    else:
        not_parsed+=1

Let's take a look at the results:

In [16]:
print(f"{citation_nr} citations found in {len(glob.glob('articles-content/*json.gz'))} articles")
print(f"{citation_url_nr} citations having a URL")
print(f"{tempname_in_spreadsheet} citations matching our spreadsheet of templates")
print(f"{tempname_in_spreadsheet_url} citations matching our spreadsheet of templates having a URL")
#print(f"Unable to parse {not_parsed} citations")


13995340 citations found in 10570 articles
9579066 citations having a URL
650776 citations matching our spreadsheet of templates
456133 citations matching our spreadsheet of templates having a URL
Unable to parse 0 citations


## Querying Citoid API

In this section, we request the [Citoid API](https://en.wikipedia.org/api/rest_v1/#/Citation/getCitation) to find the citation metadata for the list of URLs found on the previous step.

In [ ]:
dfs = []

for filename in sorted(os.listdir('citations'))[:20]: ########
    wikilang = filename.split('-')[0]

    with gzip.open(f'citations/{filename}') as fi:
        article_citations = pd.read_csv(fi)

    dfs.append(article_citations)
    
all_citations = pd.concat(dfs, ignore_index= True)
all_citations

In [ ]:
def get_and_cache_response(reference_url, outfname, HEADER):
    
    if not os.path.isfile(outfname):
        
        print(outfname, reference_url)
        parsed_url = urllib.parse.quote(reference_url).replace('/', '%2F')
        response = requests.get(
            url = 'https://en.wikipedia.org/api/rest_v1/data/citation/mediawiki/'+parsed_url,
            headers = HEADER
        )

        if response.status_code == 200:
            citoid_data = response.json()
            with gzip.open(outfname, 'w') as fo:
                fo.write(json.dumps(citoid_data).encode('utf-8'))

In [ ]:
citoid_cache_dir = 'citoid-cache/'
os.makedirs(citoid_cache_dir, exist_ok = True)

for index, row in all_citations.iterrows():
    
    pageid = all_citations.loc[index]['page_id']
    lang = all_citations.loc[index]['wiki_lang']
    outfname = f"{citoid_cache_dir}{pageid}-{lang}.json.gz"
    
    reference_url = row['url']
    get_and_cache_response(reference_url, outfname, HEADER)

## Tidying up the queried data 

Salida: tabla de URLs citadas, incluyendo
+ Número de veces citada en el corpus de ingreso
+ Tipo de fuente (art científico, cap libro, etc). >> Info del citation template
+ 4 metadatos básicos, según consta en artículo y devuelto por API Citoid
    - Título
    - Autores
    - Fecha de publicacion
    - Publicación (nombre diario, revista, nombre del sitio)
    - Tipo de fuente
+ Validar la metodología reproduciendo los datos publicados por Andrew Lih en 2017 (https://docs.google.com/spreadsheets/d/1kNHSVKq5qZr6_3-05UFtXF2RA5sn3HdUUgL0nC57XPk/edit#gid=543588472)

## Evaluation

+ Veredicto “soportado por Citoid”
+ Puntaje de coincidencia, usando distancia de edición? Suponemos metadatos del template son correctos (los suponemos curados)
+ Opcional: estado en dashboard traductores Zotero (o “sin traductor”)
+ Opcional: JSON+LD (la URL ofrece metadatos en JSON+LD, sí o no)

## Results

- Obtener la tabla de cobertura Citoid para cada URL y metadato básico
- Mostrar porcentajes de cobertura (Citoid coverage gap) por
    + Tipo de fuente
    + Idioma de la Wikipedia
    